In [ ]:
import pandas as pd

data_airport = pd.read_csv("Raw datasets/airports.csv")
data_routes = pd.read_csv("Raw datasets/routes.csv")

# Check for missing values
data_airport.isnull().sum()

# remove from airport - iata, altitude, timezone, dst, tz_database, type, source
data_airport = data_airport.drop(columns=['altitude', 'timezone', 'dst', 'tz_database', 'type', 'source'])

# Hvorfor valgte vi kun disse, for the you only need columns that identify airports and their locations. City and country are for easy identification.
pd.DataFrame(data_routes).head()
print(len(data_routes))
# For 1 stopp er det kun 11 mellomlandinger og resten som har 0 stopp har 67652
data_routes["stops"].value_counts()

data_routes.isnull().sum()
data_routes.dropna(subset=['src_airport', 'dst_airport'], inplace=True)
# Remove from routes - airline, airline_id, codeshare, equipment, stops
#data_routes = data_routes.drop(columns=['airline', 'airline_id', 'codeshare', 'equipment', 'stops', 'src_id', 'dst_id'])

# Hvorfor vi valgte kun disse, for the you only need columns that identify the source and destination airports. 
# src airport and dst airport were kept because they are readable and they have the same information as src_id and dst_id.
# id for src and dst have a lot higher count of null values, this was taken into consideration when removing them.
# we removed stops because it was only 11 routes with 1 stop, which is negligible compared to the total number of routes.

data_routes = data_routes.drop(columns=['airline', 'airline_id', 'codeshare', 'equipment', 'stops', 'src_id', 'dst_id'])




# Create sets of valid airport codes (IATA or ICAO depending on your map)
valid_codes = set(data_airport["icao"].dropna().unique()) | set(data_airport["iata"].dropna().unique())
# Identify which routes reference unknown airports
missing_src = data_routes[~data_routes["src_airport"].isin(valid_codes)]
missing_dst = data_routes[~data_routes["dst_airport"].isin(valid_codes)]

# Combine both
missing_routes = pd.concat([missing_src, missing_dst]).drop_duplicates()


print(f"Found {len(missing_routes)} routes with missing airports.")
data_routes.drop(missing_routes.index, inplace=True)

# De
print(len(data_routes))



67663
Found 553 routes with missing airports.
67027


In [12]:
# FIXED VERSION: Check routes against the correct airport identifier

import pandas as pd

airports = pd.read_csv("airports_cleaned.csv")
routes = pd.read_csv("routes_cleaned.csv")

# Routes use ICAO codes (4-letter), so create a set of valid ICAO codes
valid_codes = set(airports["icao"].dropna().unique()) | set(airports["iata"].dropna().unique())

print(f"Valid airport codes: {len(valid_codes)}")
print(f"Total routes: {len(routes)}")

# Find routes where source or destination airports are NOT in our airports list
missing_src = routes[~routes["src_airport"].isin(valid_codes)]
missing_dst = routes[~routes["dst_airport"].isin(valid_codes)]

print(f"\nRoutes with missing source airport: {len(missing_src)}")
print(f"Routes with missing destination airport: {len(missing_dst)}")

# Get unique missing airport codes
missing_src_codes = set(missing_src["src_airport"].unique())
missing_dst_codes = set(missing_dst["dst_airport"].unique())
all_missing_codes = missing_src_codes | missing_dst_codes

print(f"\nUnique missing airport codes: {len(all_missing_codes)}")
print(f"Sample missing codes: {list(all_missing_codes)[:20]}")



print(f"\n✅ Found {len(missing_routes)} routes referencing {len(all_missing_codes)} unknown airports.")


# Optional: Clean the routes by removing entries with missing airports
valid_routes = routes[
    routes["src_airport"].isin(valid_codes) & 
    routes["dst_airport"].isin(valid_codes)
]
print(f"\n✅ Valid routes (both airports exist): {len(valid_routes)}")

print(len(data_routes))



Valid airport codes: 13767
Total routes: 67027

Routes with missing source airport: 91
Routes with missing destination airport: 92

Unique missing airport codes: 34
Sample missing codes: ['MRA', 'CZH', 'WNH', 'NTG', 'RIG', 'KGX', 'HYL', 'CUK', 'WWT', 'WTL', 'PHO', 'FSZ', 'TVS', 'TLT', 'KKI', 'TNK', 'OLH', 'GSM', 'KTB', 'INC']

✅ Found 553 routes referencing 34 unknown airports.

✅ Valid routes (both airports exist): 66851
67027
